In [1]:
import os
import numpy
from scipy.sparse import csr_matrix
from mnick_rescal import als
from triple_reader import triple_reader
from question_reader import question_reader
from nltk.stem import WordNetLemmatizer
# MUST USE WORD2VEC, FIND EITHER ENTITY-ENTITY OR ENTITY-RELATION that are closest
# from gensim import utils
from gensim.models import word2vec

In [2]:
# read in file as tensors
text_file = ("/Users/SaahilM/Documents/Princeton/Academics/Thesis/"
    "Senior Thesis Code/ModifiedEntityGraph/prod/MCTest/production/MCTest/OCR_text/1/Triples/1-medium.txt")
    
tr = triple_reader(text_file)
print tr.tripleList
tensor = tr.tensor
enMap = tr.enMap
relMap = tr.relMap

[['critic' 'recalls' 'painting']
 ['novelist' 'recalls' 'painting']
 ['I' 'was' 'bought']
 ['painting' 'was' 'Fell']
 ['anyone' 'read' 'books']
 ['anyone' 'be' 'read']
 ['I' 'went' 'books']
 ['I' 'be' 'went']
 ['writers' 'need' 'books']
 ['writers' 'be' 'need']
 ['she' 'feeling' 'same']
 ['I' 'imagined' 'same']
 ['I' 'was' 'wrong']
 ['she' 'sold' 'them']
 ['she' 'sold' 'what']
 ['Sheila' 'care' 'them']
 ['Sheila' 'care' 'what']
 ['they' 'thought' 'them']
 ['they' 'thought' 'what']
 ['who' 'looked' 'them']
 ['who' 'looked' 'what']
 ['She' 'ease' 'work']
 ['She' 'was' 'ease']
 ['that' 'made' 'canvas']
 ['that' 'made' 'me']
 ['confidence' 'was' 'canvas']
 ['confidence' 'was' 'me']
 ['she' 'propped' 'canvas']
 ['she' 'propped' 'me']
 ['canvas' 'relax' 'canvas']
 ['canvas' 'relax' 'me']
 ['need' 'was' 'Cumberland']
 ['need' 'been' 'was']
 ['I' 'apprehensive' 'Cumberland']
 ['I' 'been' 'apprehensive']
 ['I' 'know' 'Cumberland']
 ['I' 'been' 'know']
 ['we' 'had' 'Cumberland']
 ['we' 'been' 'h

In [3]:
print len(tensor)
print tensor[0].shape
# convert them to scipy csr_matrix
csrTensor = map(lambda x: csr_matrix(x), tensor)

A, R, f, itr, exectimes = als(csrTensor, 20)

print A
print R
print f
print itr
print exectimes

46
(81, 81)
[[  3.45845077e-02  -3.45392908e-02  -1.55105896e-03 ...,   5.07464718e-02
    1.10360939e-02   1.82443658e-02]
 [ -3.22935261e-02  -1.85661604e-02   2.65538710e-02 ...,   2.33040279e-02
   -6.88325252e-02   1.22067408e-02]
 [ -3.85973601e-02  -9.86483921e-02  -3.92504419e-02 ...,   1.41543710e-01
   -6.12612141e-03   8.11542262e-02]
 ..., 
 [ -1.39071467e-01   2.92271638e-03  -4.40826147e-01 ...,  -6.43540279e-01
    7.57934220e-03   7.89986540e-02]
 [ -4.77073004e-04  -2.65582731e-03   5.84020713e-03 ...,   2.70164562e-03
   -3.33599778e-03   1.62950946e-03]
 [ -1.40496137e-02  -3.86986911e-02   1.12237964e-02 ...,   9.78260879e-03
   -1.58405654e-02   3.51015597e-02]]
[array([[ -4.98557462e-04,  -9.81934443e-04,   5.19722959e-03,
         -4.55174477e-03,   3.65419550e-04,  -1.04519476e-02,
          2.03424668e-03,   6.55006880e-14,  -9.21758590e-14,
          7.25249841e-05,  -8.60640157e-14,   2.50675607e-14,
         -7.66877007e-04,   1.25191275e-03,   3.64865166e-0

In [4]:
# initialize word2vec model
word_model = word2vec.Word2Vec.load_word2vec_format('word2vec.bin', binary=True)


In [40]:
# Find the word in entity that's most similar to given word
def findSimEn(word):
    topsim = None
    topEn = None
    for en in enMap:
        if type(en) == int:
            continue
        sim = word_model.similarity(en, word)
        if sim > topsim:
            topsim = sim
            topEn = en
    return [topEn, topsim]

def findSimRel(word):
    topsim = None
    topRel = None
    for rel in relMap:
        if type(rel) == int:
            continue
        sim = word_model.similarity(rel, word)
        if sim > topsim:
            topsim = sim
            topRel = rel
    return [topRel, topsim]

print findSimEn('critic')
print findSimRel('go')

# Return top similarity, sim score, and isEn boolean
def findTopEnOrRel(word):
    [topEn, topEnsim] = findSimEn(word)
    [topRel, topRelsim] = findSimRel(word)
    if topEnsim > topRelsim:
        return (topEn, topEnsim, True, word)
    else:
        return (topRel, topRelsim, False, word)
    
# Return top 2 sims for an array of words
def findTopEnOrRelArr(wordArr):
    topArrs = []
    for word in wordArr:
        top = findTopEnOrRel(word)
#         print top
        topArrs.append(top)
    print topArrs
    sortedTop = sorted(topArrs, key=lambda x:-x[1])
    print sortedTop
    top1 = sortedTop[0]
    top2 = None
    # if the top is a relation, we have to pick an entity
    if top1[2] == False:
        curIndex = 1
        while curIndex < len(sortedTop):
            print "curIndex ", curIndex
            cur = sortedTop[curIndex]
            if cur[2] == False:
                curIndex += 1
                continue
            else:
                top2 = cur
                break
    else:
        top2 = sortedTop[1]
    if top2 == None:
        top2 = [0, 0, True]
    return [top1, top2]

print findTopEnOrRelArr(["critic", "went", "hi", "boat", "paint"])

['critic', 1.0000000000000002]
['went', 0.56464235044689293]
[('critic', 1.0000000000000002, True, 'critic'), ('went', 0.99999999999999989, False, 'went'), ('let', 0.29922765246074712, True, 'hi'), ('house', 0.32290397163052165, True, 'boat'), ('painting', 0.53587220658339196, False, 'paint')]
[('critic', 1.0000000000000002, True, 'critic'), ('went', 0.99999999999999989, False, 'went'), ('painting', 0.53587220658339196, False, 'paint'), ('house', 0.32290397163052165, True, 'boat'), ('let', 0.29922765246074712, True, 'hi')]
[('critic', 1.0000000000000002, True, 'critic'), ('went', 0.99999999999999989, False, 'went')]


In [61]:
# for each question choice, find the entity/relation that is most-matched in question 
# then we find the choice that matches the result the most

q_file = ("/Users/SaahilM/Documents/Princeton/Academics/Thesis/Senior Thesis Code/"
"ModifiedEntityGraph/prod/MCTest/production/MCTest/OCR_text/1/1-medium1-q.txt")
qr = question_reader(q_file)
questions = qr.questionCombos

for question in questions:
    question_words = question[0].split(" ")
    print question_words
    print type(question_words[0])
    print map(lambda x: x, question_words)
#     print str(question_words)
    [top1, top2] = findTopEnOrRelArr(question_words)
    print [top1, top2]

['\x00I\x00n\x00', '\x00l\x00i\x00n\x00e\x00', '\x002\x005\x00,\x00', '\x00t\x00h\x00e\x00', '\x00a\x00u\x00t\x00h\x00o\x00r\x00', '\x00a\x00s\x00s\x00u\x00m\x00e\x00s\x00', '\x00t\x00h\x00a\x00t\x00', '\x00"\x00j\x00u\x00s\x00t\x00i\x00c\x00e\x00"\x00', '\x00w\x00o\x00u\x00l\x00d\x00', '\x00b\x00e\x00', '\x00', '\x00', '\x00', '\x00']
<type 'str'>
['\x00I\x00n\x00', '\x00l\x00i\x00n\x00e\x00', '\x002\x005\x00,\x00', '\x00t\x00h\x00e\x00', '\x00a\x00u\x00t\x00h\x00o\x00r\x00', '\x00a\x00s\x00s\x00u\x00m\x00e\x00s\x00', '\x00t\x00h\x00a\x00t\x00', '\x00"\x00j\x00u\x00s\x00t\x00i\x00c\x00e\x00"\x00', '\x00w\x00o\x00u\x00l\x00d\x00', '\x00b\x00e\x00', '\x00', '\x00', '\x00', '\x00']


KeyError: '\x00I\x00n\x00'